In [67]:
from datetime import datetime, timedelta

In [68]:
hospital_data = {
    'patients' : {},
    'medical_staff' : {},
    'appointments': [],      
    'medical_records': [],
    'billing': {},            
    'emergency_admissions': [], 
    'medications': {},        
    'rooms': {},              
    'equipment': {},          
    'discharges': {},     
}

def register_patient(patient_id, personal_info, medical_history, insurance_info):
    if patient_id in hospital_data['patients']:
        print("Patient already Registered")
        return
    hospital_data['patients'][patient_id] = {
        'personal_info': personal_info,
        'medical_history': medical_history,
        'insurance_info': insurance_info,
        'appointments': [],
        'status': 'Outpatient',
        'current_room': None,
        'admission_date': None,
        'discharge_date': None,
        'follow_up_instructions': None
    }
    print(f"Patient {personal_info['name']} Registered Successfully.")

def add_medical_staff(staff_id, name, specialization, shift_schedule, contact_info):
    if staff_id in hospital_data['medical_staff']:
        print("Staff already Registered.")
        return
    hospital_data['medical_staff'][staff_id] = {
        'name': name,
        'specialization': specialization,
        'shift_schedule': shift_schedule,
        'contact_info': contact_info,
    }
    print(f"Medical staff {name} added.")

def schedule_appointment(patient_id, doctor_id, appointment_date, appointment_type):
    if patient_id not in hospital_data['patients'] or doctor_id not in hospital_data['medical_staff']:
        print("Invalid patient or doctor ID.")
        return
    if isinstance(appointment_date, str):
        try:
            appointment_date = datetime.strptime(appointment_date, "%Y-%m-%d %H:%M")
        except ValueError:
            print("Incorrect date format! Use YYYY-MM-DD HH:MM")
            return
    for appt in hospital_data['appointments']:
        appt_date = appt['appointment_date']
        if isinstance(appt_date, str):
            appt_date = datetime.strptime(appt_date, "%Y-%m-%d %H:%M")
        if (appt['doctor_id'] == doctor_id or appt['patient_id'] == patient_id) and appt_date == appointment_date: # D100 P1234 appt_date =  
            print("Appointment conflict detected.")
            return
    appointment = {
        'appointment_id': f"APT{len(hospital_data['appointments']) + 1:05d}", # 7+1 = 8  APT00008
        'patient_id': patient_id,
        'doctor_id': doctor_id,
        'appointment_date': appointment_date.strftime("%Y-%m-%d %H:%M"),
        'appointment_type': appointment_type,
    }
    hospital_data['appointments'].append(appointment)
    hospital_data['patients'][patient_id]['appointments'].append(appointment)
    print(f"Appointment scheduled for {hospital_data['patients'][patient_id]['personal_info']['name']} with {hospital_data['medical_staff'][doctor_id]['name']} on {appointment['appointment_date']}.")

def create_medical_record(patient_id, doctor_id, diagnosis, treatment, prescription):
    record = {
        'record_id': f"REC{len(hospital_data['medical_records']) + 1:05d}",
        'patient_id': patient_id,
        'doctor_id': doctor_id,
        'diagnosis': diagnosis,
        'treatment': treatment,
        'prescription': prescription,
        'date': datetime.now().strftime("%Y-%m-%d %H:%M"),
    }
    hospital_data['medical_records'].append(record)
    print(f"Medical record created for patient {patient_id} by doctor {doctor_id}.")

def process_billing(patient_id, services_list, insurance_coverage):
    total = sum(cost for (_, cost) in services_list)
    coverage_amount = total * (insurance_coverage / 100.0)
    due = total - coverage_amount
    bill = {
        'services': services_list,
        'total': total,
        'insurance_coverage': coverage_amount,
        'patient_due': due,
        'date': datetime.now().strftime("%Y-%m-%d %H:%M"),
    }
    # hospital_data['billing'][patient_id].append(bill)
    hospital_data['billing'][patient_id]=bill
    print(f"Billing processed: Total = {total}, Covered = {coverage_amount}, Due = {due}")

def manage_emergency_admission(patient_id, emergency_type, severity_level):
    if patient_id not in hospital_data['patients']:
        register_patient(
            patient_id,
            {'name': 'Unknown', 'age': None, 'gender': None, 'blood_type': None},
            [],
            {}
        )
    admission = {
        'patient_id': patient_id,
        'emergency_type': emergency_type,
        'severity_level': severity_level,
        'admitted_on': datetime.now().strftime("%Y-%m-%d %H:%M"),
    }
    hospital_data['emergency_admissions'].append(admission)
    print(f"Emergency admission for {patient_id} ({emergency_type}, severity {severity_level})")

    assigned = False
    for room_type in ['Emergency', 'General']:
        for room_id, room in hospital_data['rooms'].items():
            if room['room_type'] == room_type and not room['occupied']:
                assign_room(patient_id, room_type, admission_date=datetime.now(), expected_duration=3)
                assigned = True
                break
        if assigned:
            break
    if not assigned:
        print("No Emergency or General room available for this emergency admission.")


def track_medication_inventory(medication_id, quantity, expiry_date, supplier):

    if isinstance(expiry_date, datetime):
        expiry_str = expiry_date.strftime("%Y-%m-%d")
    else:
        expiry_str = expiry_date
    hospital_data['medications'][medication_id] = {
        'quantity': quantity,
        'expiry_date': expiry_str,
        'supplier': supplier
    }
    print(f"Medication {medication_id} updated. Quantity: {quantity}")

def assign_room(patient_id, room_type, admission_date=None, expected_duration=3):
    if admission_date is None:
        admission_date = datetime.now()
    elif isinstance(admission_date, str):
        admission_date = datetime.strptime(admission_date, "%Y-%m-%d")
    
    for room_id, room in hospital_data['rooms'].items():
        if room['room_type'] == room_type and not room['occupied']:
            room['occupied'] = True
            room['patient_id'] = patient_id
            hospital_data['patients'][patient_id]['current_room'] = room_id
            hospital_data['patients'][patient_id]['status'] = 'Inpatient'
            hospital_data['patients'][patient_id]['admission_date'] = admission_date.strftime("%Y-%m-%d")
            hospital_data['patients'][patient_id]['expected_discharge'] = (admission_date + timedelta(days=expected_duration)).strftime("%Y-%m-%d")
            print(f"Assigned room {room_id} to patient {patient_id}")
            return
    print("No available room of requested type.")

def manage_discharge_process(patient_id, discharge_date=None, follow_up_instructions=None):
    patient = hospital_data['patients'].get(patient_id)

    if not patient:
        print(f"No patient found with ID {patient_id}")
        return

    if discharge_date is None:
        discharge_date = datetime.now()
    elif isinstance(discharge_date, str):
        for fmt in ("%Y-%m-%d %H:%M:%S", "%Y-%m-%d %H:%M"):
            try:
                discharge_date = datetime.strptime(discharge_date, fmt)
                break
            except ValueError:
                continue
        else:
            print("Error: Discharge date format should be 'YYYY-MM-DD HH:MM[:SS]'")
            return

    patient['discharge_date'] = discharge_date.strftime("%Y-%m-%d %H:%M:%S")
    patient['follow_up_instructions'] = follow_up_instructions
    patient['status'] = 'Discharged'

    print(f"Patient {patient['personal_info']['name']} (ID: {patient_id}) discharged on {patient['discharge_date']}.")
    if follow_up_instructions:
        print("Follow-up Instructions:", follow_up_instructions)
    
    room_id = patient.get('current_room')
    if room_id and room_id in hospital_data['rooms']:
        hospital_data['rooms'][room_id]['occupied'] = False
        hospital_data['rooms'][room_id]['patient_id'] = None
        print(f"Room {room_id} is now unoccupied.")


In [69]:
def calculate_treatment_cost(patient_id):
    if patient_id not in hospital_data['patients']:
        print(f"Patient {patient_id} not found.")
        return

    if patient_id not in hospital_data['billing']:
        print(f"No billing record found for {patient_id}.")
        return

    services_list = hospital_data['billing'][patient_id]['services']

    treatment_plan = [{'service': s[0], 'cost': s[1]} for s in services_list]

    insurance_details = hospital_data['patients'][patient_id]['insurance_info']
    coverage = insurance_details.get('coverage', 0)

    total_cost = sum(item['cost'] for item in treatment_plan)
    covered_amount = (coverage / 100) * total_cost
    patient_payable = total_cost - covered_amount

    hospital_data['billing'][patient_id].update({
        'treatment_plan': treatment_plan,
        'total_cost': total_cost,
        'insurance_provider': insurance_details.get('provider'),
        'coverage_percent': coverage,
        'covered_amount': covered_amount,
        'patient_payable': patient_payable,
        'date': datetime.now().strftime("%Y-%m-%d %H:%M")
    })

    print(f"=== TREATMENT COST SUMMARY ===")
    print(f"Patient ID: {patient_id}")
    print(f"Total Treatment Cost: {total_cost}")
    print(f"Insurance Coverage ({coverage}%): -{covered_amount}")
    print(f"Amount Payable by Patient: {patient_payable}\n")

    return patient_payable



def analyze_hospital_efficiency(metrics_type, time_period):

    start = datetime.strptime(time_period[0], "%Y-%m-%d")
    end = datetime.strptime(time_period[1], "%Y-%m-%d")
    days = (end - start).days + 1

    if metrics_type == 'bed_occupancy':
        occupied_beds = 0
        for patient_id, patient in hospital_data['patients'].items():
            adm_date = patient.get('admission_date')
            dis_date = patient.get('discharge_date')

            if adm_date:
                adm_date_dt = datetime.strptime(adm_date, "%Y-%m-%d")
                if dis_date:
                    for fmt in ("%Y-%m-%d %H:%M:%S", "%Y-%m-%d %H:%M", "%Y-%m-%d"):
                        try:
                            dis_date_dt = datetime.strptime(dis_date, fmt)
                            break
                        except ValueError:
                            continue
                    else:
                        print(f"Invalid discharge date format for patient {patient_id}: {dis_date}")
                        continue
                else:
                    dis_date_dt = datetime.now()

                if adm_date_dt <= end and dis_date_dt >= start:
                    occupied_beds += 1

        avg_occupancy = occupied_beds / days if days > 0 else 0
        print(f"Bed Occupancy Rate: {avg_occupancy:.2f} beds/day\n")
        return avg_occupancy

    elif metrics_type == 'average_stay':
        total_days = 0
        count = 0
        for patient_id, patient in hospital_data['patients'].items():
            adm_date = patient.get('admission_date')
            dis_date = patient.get('discharge_date')

            if adm_date and dis_date:
                adm_date_dt = datetime.strptime(adm_date, "%Y-%m-%d")
                dis_date_dt = datetime.strptime(dis_date.split()[0], "%Y-%m-%d")  # Split if time included
                stay = (dis_date_dt - adm_date_dt).days + 1
                total_days += stay
                count += 1

        avg_stay = total_days / count if count > 0 else 0
        print(f"Average Length of Stay: {avg_stay:.2f} days\n")
        return avg_stay

    else:
        print("Invalid metrics type.\n")
        return None

def generate_patient_report(patient_id, report_type):
    
    #Generates patient reports: 'summary', 'medical_history', or 'billing'.
    
    patient = hospital_data['patients'].get(patient_id)
    if not patient:
        print(f"Patient {patient_id} not found.")
        return

    print(f"=== PATIENT REPORT: {report_type.upper()} ===")
    print(f"Name: {patient['personal_info']['name']} | ID: {patient_id}")

    if report_type == 'summary':
        print(f"Age: {patient['personal_info'].get('age','N/A')} | Gender: {patient['personal_info'].get('gender','N/A')}")
        print(f"Blood Type: {patient['personal_info'].get('blood_type','N/A')}")
        print(f"Status: {patient.get('status','N/A')}")
        print(f"Current Room: {patient.get('current_room','N/A')}")
        print(f"Admission Date: {patient.get('admission_date','N/A')}")
        print(f"Discharge Date: {patient.get('discharge_date','N/A')}")

    elif report_type == 'medical_history':
        if patient['medical_history']:
            for record in patient['medical_history']:
                if isinstance(record, dict):
                    print(f"- {record.get('date','N/A')}: {record.get('diagnosis','N/A')} | Treatment: {record.get('treatment','N/A')}")
                else:
                    print(f"- {record}")
        else:
            print("No medical history available.")

    elif report_type == 'billing':
        billing = hospital_data['billing'].get(patient_id)
        if billing:
            print(f"Total Cost: {billing.get('total_cost','N/A')}")
            print(f"Insurance Provider: {billing.get('insurance_provider','N/A')} ({billing.get('coverage_percent',0)}%)")
            print(f"Covered Amount: {billing.get('covered_amount','N/A')}")
            print(f"Patient Payable: {billing.get('patient_payable','N/A')}")
            print(f"Date: {billing.get('date','N/A')}")
        else:
            print("No billing record found.")

    else:
        print("Invalid report type.\n")



In [70]:
# 1st Floor Rooms: 101-A to 105-E
hospital_data['rooms']['101-A'] = {'room_type': 'General', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['102-B'] = {'room_type': 'ICU', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['103-C'] = {'room_type': 'Private', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['104-D'] = {'room_type': 'Deluxe', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['105-E'] = {'room_type': 'Emergency', 'occupied': False, 'patient_id': None}

# 2nd Floor Rooms: 201-A to 205-E
hospital_data['rooms']['201-A'] = {'room_type': 'General', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['202-B'] = {'room_type': 'ICU', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['203-C'] = {'room_type': 'Private', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['204-D'] = {'room_type': 'Deluxe', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['205-E'] = {'room_type': 'General', 'occupied': False, 'patient_id': None}

# 3rd Floor Rooms: 301-A to 305-E
hospital_data['rooms']['301-A'] = {'room_type': 'General', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['302-B'] = {'room_type': 'ICU', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['303-C'] = {'room_type': 'Private', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['304-D'] = {'room_type': 'Deluxe', 'occupied': False, 'patient_id': None}
hospital_data['rooms']['305-E'] = {'room_type': 'General', 'occupied': False, 'patient_id': None}

In [71]:
register_patient('P12345', {'name': 'Rajesh Kumar', 'age': 45, 'gender': 'Male', 'blood_type': 'B+'}, ['Hypertension'], {'provider': 'Star Health', 'policy': 'SH789456', 'coverage' : 80})
register_patient('P12346', {'name': 'Bhavan Naggam', 'age': 20, 'gender': 'Others', 'blood_type': 'B+'}, ['Shoulder Dislocation'], {'provider': 'Star Health', 'policy': 'SH789489', 'coverage' : 70})

add_medical_staff('D100', 'Dr. Priya Mehta', 'Cardiology', [('08:00', '16:00')], {'phone': '9876543210'})
add_medical_staff('D101', 'Dr. Sanjay Gupta', 'Ortho', [('09:00', '16:00')], {'phone': '8765940213'})

schedule_appointment('P12345', 'D100', '2025-03-18 10:00', 'ECG Review')
schedule_appointment('P12346', 'D101', '2025-03-18 10:00', 'X-Ray Review')

create_medical_record('P12345','D100','Hypertension','Lifestyle changes, antihypertensive medication','Metoprolol 50mg twice daily')
create_medical_record('P12346','D101','Shoulder Dislocation','Excercise, Physio Therapy','Calicum Tablets 50mg one daily')

calculate_treatment_cost('P12345')

assign_room('P12345', 'General', '2025-03-15', 3)

manage_emergency_admission('P99999', 'Heart Attack', 'Critical')

process_billing('P12345', [("Room Charges (3 days)", 4500), ("Doctor Consultation", 2000), ("ECG Test", 800), ("Blood Tests", 1200), ("Medications", 1800), ("Nursing Care", 2400)], 80)

track_medication_inventory('MED001', 50,'2025-12-31','Apollo Pharma')
track_medication_inventory('MED003',120,'2025-09-30','Cipla Ltd')

manage_discharge_process("P12345", discharge_date="2025-03-18 15:30:00", follow_up_instructions="Routine check after 1 week.")
manage_discharge_process("P12346", follow_up_instructions="Visit Ortho after 15 days.")

generate_patient_report('P12345', 'summary')
generate_patient_report('P12345', 'medical_history')
generate_patient_report('P12345', 'billing')

analyze_hospital_efficiency('bed_occupancy', ('2025-06-01', '2025-06-30'))
analyze_hospital_efficiency('average_stay', ('2025-06-01', '2025-06-30'))




Patient Rajesh Kumar Registered Successfully.
Patient Bhavan Naggam Registered Successfully.
Medical staff Dr. Priya Mehta added.
Medical staff Dr. Sanjay Gupta added.
Appointment scheduled for Rajesh Kumar with Dr. Priya Mehta on 2025-03-18 10:00.
Appointment scheduled for Bhavan Naggam with Dr. Sanjay Gupta on 2025-03-18 10:00.
Medical record created for patient P12345 by doctor D100.
Medical record created for patient P12346 by doctor D101.
No billing record found for P12345.
Assigned room 101-A to patient P12345
Patient Unknown Registered Successfully.
Emergency admission for P99999 (Heart Attack, severity Critical)
Assigned room 105-E to patient P99999
Billing processed: Total = 12700, Covered = 10160.0, Due = 2540.0
Medication MED001 updated. Quantity: 50
Medication MED003 updated. Quantity: 120
Patient Rajesh Kumar (ID: P12345) discharged on 2025-03-18 15:30:00.
Follow-up Instructions: Routine check after 1 week.
Room 101-A is now unoccupied.
Patient Bhavan Naggam (ID: P12346) d

4.0

In [72]:
def print_patient_dashboard(patient_id, doctor_id=None):
    patient = hospital_data['patients'].get(patient_id)
    if not patient:
        print("Patient not found.")
        return

    doctor_name = specialization = "N/A"
    if doctor_id and doctor_id in hospital_data['medical_staff']:
        doctor = hospital_data['medical_staff'][doctor_id]
        doctor_name = doctor['name']
        specialization = doctor['specialization']

    ins = patient['insurance_info']
    insurance_str = f"{ins.get('provider','N/A')} (Policy: {ins.get('policy','N/A')})"

    room_id = patient.get('current_room') or "N/A"
    room_type = hospital_data['rooms'][room_id]['room_type'] if room_id in hospital_data['rooms'] else "N/A"

   
    admission_date = patient.get('admission_date') or "N/A"
    discharge_date = patient.get('discharge_date') or "N/A"

    # Dummy Vitals
    vitals = {
        "date": "N/A",
        "Blood Pressure": "N/A",
        "Heart Rate": "N/A",
        "Temperature": "N/A",
        "Oxygen Saturation": "N/A",
    }

    meds = []
    medical_records = [
        rec for rec in hospital_data['medical_records'] if rec['patient_id'] == patient_id
    ]
    if medical_records:
        # most recent record by date
        def parse_date(date_str):
            for fmt in ("%Y-%m-%d %H:%M", "%Y-%m-%d %H:%M:%S"):
                try:
                    return datetime.strptime(date_str, fmt)
                except:
                    continue
            return datetime.min
        medical_records.sort(key=lambda r: parse_date(r.get('date', '')), reverse=True)
        latest_record = medical_records[0]
        meds = latest_record.get('prescription', [])
        if isinstance(meds, str):
            meds = [meds]  # In case prescription is stored as a string, not a list

    # --- PATIENT DASHBOARD ---
    print("=== PATIENT DASHBOARD ===")
    print(f"Patient: {patient['personal_info']['name']} (ID: {patient_id})")
    print(f"Age: {patient['personal_info']['age']} | Gender: {patient['personal_info']['gender']} | Blood Type: {patient['personal_info']['blood_type']}")
    print(f"Insurance: {insurance_str}\n")
    print(f"Current Status: {patient['status']}")
    print(f"Room: {room_id} ({room_type})")
    print(f"Admitted: {admission_date}")
    print(f"Discharged: {discharge_date}")
    print(f"Attending Doctor: {doctor_name} ({specialization})\n")
    print(f"Recent Vitals ({vitals['date']}):")
    print(f"Blood Pressure: {vitals['Blood Pressure']}")
    print(f"Heart Rate: {vitals['Heart Rate']}")
    print(f"Temperature: {vitals['Temperature']}")
    print(f"Oxygen Saturation: {vitals['Oxygen Saturation']}\n")
    print("Active Medications:")
    if meds:
        for m in meds:
            print(f"- {m}")
    else:
        print("None")


def print_appointment_schedule(patient_id=None, doctor_id=None):
    print("=== APPOINTMENT SCHEDULE ===")
    for appt in hospital_data['appointments']:
        if patient_id and appt['patient_id'] == patient_id:
            doc = hospital_data['medical_staff'][appt['doctor_id']]
            print(f"{appt['appointment_date']} - {appt['appointment_type']} - {doc['name']}")
        elif doctor_id and appt['doctor_id'] == doctor_id:
            pat = hospital_data['patients'][appt['patient_id']]
            print(f"{appt['appointment_date']} - {appt['appointment_type']} - {pat['personal_info']['name']}")

    if not patient_id and not doctor_id:
        for appt in hospital_data['appointments']:
            doc = hospital_data['medical_staff'][appt['doctor_id']]
            pat = hospital_data['patients'][appt['patient_id']]
            print(f"{appt['appointment_date']} - {appt['appointment_type']} - Patient: {pat['personal_info']['name']} - Doctor: {doc['name']}")

def print_billing_summary(patient_id):
    patient = hospital_data['patients'].get(patient_id)
    if not patient:
        print("Patient not found.")
        return
    bill = hospital_data['billing'].get(patient_id)
    admission_date = "March 15, 2025"
    discharge_date = "March 18, 2025"
    print("=== BILLING SUMMARY ===")
    if bill:
        print(f"Patient: {patient['personal_info']['name']}")
        print(f"Admission Date: {admission_date}")
        print(f"Discharge Date: {discharge_date}\n")
        print("Service Charges:")
        for service, cost in bill['services']:
            print(f"{service}: ₹{cost}")
        print(f"\nTotal Bill: ₹{bill['total']}")
        print(f"Insurance Coverage (80%): ₹{int(bill['insurance_coverage'])}")
        print(f"Patient Responsibility: ₹{int(bill['patient_due'])}")
    else:
        print("No billing information available.")


In [73]:
# print_patient_dashboard("P12345")
print_patient_dashboard("P12345", doctor_id="D100")

# print_appointment_schedule(patient_id="P12345")r
# print_appointment_schedule(doctor_id="D1002")

print_appointment_schedule(patient_id="P12345", doctor_id="D1002")

print_billing_summary("P12345")

=== PATIENT DASHBOARD ===
Patient: Rajesh Kumar (ID: P12345)
Age: 45 | Gender: Male | Blood Type: B+
Insurance: Star Health (Policy: SH789456)

Current Status: Discharged
Room: 101-A (General)
Admitted: 2025-03-15
Discharged: 2025-03-18 15:30:00
Attending Doctor: Dr. Priya Mehta (Cardiology)

Recent Vitals (N/A):
Blood Pressure: N/A
Heart Rate: N/A
Temperature: N/A
Oxygen Saturation: N/A

Active Medications:
- Metoprolol 50mg twice daily
=== APPOINTMENT SCHEDULE ===
2025-03-18 10:00 - ECG Review - Dr. Priya Mehta
=== BILLING SUMMARY ===
Patient: Rajesh Kumar
Admission Date: March 15, 2025
Discharge Date: March 18, 2025

Service Charges:
Room Charges (3 days): ₹4500
Doctor Consultation: ₹2000
ECG Test: ₹800
Blood Tests: ₹1200
Medications: ₹1800
Nursing Care: ₹2400

Total Bill: ₹12700
Insurance Coverage (80%): ₹10160
Patient Responsibility: ₹2540
